# Problem 1

(a) Find the shortest vector in the two-dimensional lattice spanned by the vectors

$$
v_1=[1088186680264738983636955248541576698, -729720705100380651889073899117135412]
$$

and 
$$
v_2=[18559452608263687675065773674265003821, -12445674156096472179671403771685255651]
$$ 
 
Explain how you know your vector is the shortest, and express it as an integer combination of $v_1$ and $v_2$.

(b) Implement the LLL algorithm as described in the text, and verify that you get the same answer as the text does when applying it to the basis

$$
[19, 2, 32, 46, 3, 33],
[15, 42, 11, 0, 3, 24],
[43, 15, 0, 24, 4, 16],
[20, 44, 44, 0, 18, 15],
[0, 48, 35, 16, 31, 31],
[48, 33, 32, 9, 1, 29]
$$


In [5]:
import math

# Problem a

# This function implements the standard method of taking a dot product
# I created a default argument for the second vector so that I can only pass in one vector and get the length of it
def dot_product(vtr1,vtr2=0):
    # Checking if the second vector is equal to the default value, and if so, it sets the second vector equal to
    # the first so that the length of the first vector squared is returned
    if vtr2 == 0:
        vtr2 = vtr1
    # Asserting vector lengths are equal
    assert len(vtr1)==len(vtr2), "Vectors lengths not equal"
    # Calculating length of vectors
    length = len(vtr1)
    # Defining product variable
    product = 0
    # Iterating over the length of the vectors
    for i in range(length):
        # Adding the corresponding indexes multiplied by each other to the sum variable
        product += vtr1[i] * vtr2[i]
    # Returning sum
    return product

# Creating a method to subtract vectors with a given coefficient
def sub_vtr(vtr1, vtr2, m):
    # Asserting vector lengths are equal
    assert len(vtr1)==len(vtr2), "Vectors lengths not equal"
    # Calculating the length of the vectors
    length = len(vtr1)
    # Defining a result variable to store the subtraction
    result = []
    # Iterating over the length of the vectors
    for i in range(length):
        # Appending indexed position of vector 1 from the indexed position of vector 2 multiplied by the coefficient
        # Appended to the same index in the result vector
        result.append(vtr1[i] - m*vtr2[i])
    # Returning result vector
    return result
    
# This function implements the Gaussian Lattice Reduction method in the 2ND Dimension, exactly as implemented in the text
def gaussian(vtr1, vtr2):
    # Finding length of both vectors
    # This will return length squared but we know that the length will always be greater than 1 so comparing the 
    # squared lengths will still hold true in the inequality
    l1 = dot_product(vtr1)
    l2 = dot_product(vtr2)
    # Comparing lengths
    if l2 < l1:
        # If length of vector 2 is shorter than vector 1, swap the two vectors
        vtr1, vtr2 = vtr2, vtr1
    # Calculating m by taking the dot product of the two vectors divided by the length squared of vector 1
    # Rounding this value so that we stay in the integer lattice
    m = round(dot_product(vtr1,vtr2)/dot_product(vtr1))
    # If our resulting m == 0, return the two vectors
    if m == 0:
        return vtr1, vtr2
    else:
        # Otherwise, we define our vector 2 to be v2 = v2 - mv1
        new_vtr2 = sub_vtr(vtr2, vtr1, m)
        # Recursively calling the function again with our original vector 1 and our new vector 2
        return gaussian(vtr1, new_vtr2)
    
print("Problem a:")
v1 = [1088186680264738983636955248541576698, -729720705100380651889073899117135412]
v2 = [18559452608263687675065773674265003821, -12445674156096472179671403771685255651]
b1,b2 = gaussian(v1, v2)
m = dot_product(b1,b2)/dot_product(b1)
print("b1 =", b1)
print("b2 =", b2)
print("Final m value :", m, "≈", round(m))

# In order to show our v1 vector is the smallest vector in L, we need to first 
# show that (v1 * v2) / len(v1)^2 <= 1/2.
# We know this would need to be less than or equal to 1/2 as our resulting m was rounded to zero,
# therefore it would have had to have an original value that was less than 1/2.
# We know this should be true as len(v2) >= len(v1) by the conditions we set in our function
print("|v1 * v2| / ||v1||^2 =", m, "<= 1/2")
# We know that this is the shortest vector in the lattice as all latice vectors can be represented by
# v = av1 + bv2. We see that the length of this vector is 
# ||v||^2 = ||av1 + bv2||^2 = a^2*||v1||^2 + 2ab*(v1*v2) + b^2*||v2||^2
# We know this ||v||^2 value would necessarily have to be greater than or equal to 
# ||av1 + bv2||^2 >= a^2*||v1||^2 - 2|ab|*(v1*v2) + b^2*||v2||^2 by absolute value laws
# ||av1 + bv2||^2 >= a^2*||v1||^2 - |ab|*||v1||^2 + b^2*||v2||^2 by what was shown above:
# the fact that (v1 * v2) / len(v1)^2 <= 1/2
# We also note that ||av1 + bv2||^2 >= a^2*||v1||^2 - |ab|*||v1||^2 + b^2*||v1||^2 because ||v2|| > ||v1||
# We see now that ||av1 + bv2||^2 >= (a^2 - |ab| + b^2) * ||v1||^2
# We see that a and b are not both 0 therefore we can conclude ||v|| >= ||v1|| for all v in L
# By this we can conclude that v1 must necessarily be the smallest vector in L


# Problem b

# Function to implement the standard method of computing Gram Schmidt
def gramSchmidt(basis, index):
    # Initializing our variable to hold our orthogonal vectors
    orthogonals = []
    # Iterating over the index+1 because the index passed in uses 0-indexing
    # This is because we only want to compute values up to the index
    for i in range(index+1):
        # Setting our temporary variable to be the corresponding vector in our basis
        tmp = basis[i]
        # Iterating over the set of orthogonal vectors
        # If this set is empty, this will be skipped. This is important because on our first iteration,
        # we simply set our first vector in the orthogonal set to be the first vector in our basis
        for o_vector in orthogonals:
            # Creating the coefficient needed to multiply the corresponding orthogonal vector
            # This is calculated by the dot product of the corresponding orthogonal vector and the indexed basis vector
            # Then divided by the length squared of the orthogonal vector
            coefficient = dot_product(o_vector, tmp)/(dot_product(o_vector))
            # Subtracting the orthogonal vector multiplied by the coefficient by from our indexed basis vector
            tmp = sub_vtr(tmp, o_vector, coefficient)
        # After iteration, we append our resulting vector to our set of orthogonal vectors
        orthogonals.append(tmp)
    # Returning the orthogonal vectors
    return orthogonals

# Simplistic function to iterate over the range 2 -> number and check if each of those indexes divides the number
def simple_factorer(number):
    # Initializing factor array
    factors = []
    # Iterating over the range 2 -> number 
    for factor in range(2,number+1):
        # Checking if our current index divides our number
        if number % factor == 0:
            # If so, append the factor to our factor list
            factors.append(factor)
    # Return our found factors
    return factors

# Somewhat unneccessary function to reduce each row of our matrix by coefficient
# Not needed so I won't explain
def reducedBasis(basis):
    for index in range(len(basis)):
        vector = basis[index]
        factors = simple_factorer(vector[0])
        for factor in factors:
            f = factor
            truth = 1
            for i in range(len(vector)):
                if vector[i] % f != 0:
                    truth = 0
            if truth:
                for i in range(len(vector)):
                    vector[i] = int(vector[i]/f)
        basis[index] = vector
    return basis

# Function exactly as supplied in the text to implement the LLL algorithm
def LLL(basis):
    # Setting our result basis to be our initially passed in basis
    reduced = basis
    # Setting k to be 1 instead of 2 as python uses 0-based indexing
    k = 1
    # Creating our orthogonal set up to index k
    orthogonals = gramSchmidt(reduced, k)
    # Computing length of our vectors
    length = len(basis)
    # Creating a while loop to check if k is greater than length-1
    # This is used instead of length because of 0-based indexing
    while k<=(length-1):
        # Creating a new orthogonal set up to index k on each iteration
        orthogonals = gramSchmidt(reduced, k)
        # Setting j to k-1
        j = k-1
        # Iterating until j is less than 0
        while j>= 0:
            # Setting m (our coefficient) to be the dot product of the kth vector in the basis by the jth vector
            # in our orthogonal set. We then round this result so that we may remain in the lattice
            m = round(dot_product(reduced[k], orthogonals[j])/dot_product(orthogonals[j]))
            # Setting the kth vector in our basis to be this vector subtracted from the jth vector times m in the
            # same basis
            reduced[k] = sub_vtr(reduced[k], reduced[j], m)
            # Decrementing j
            j -= 1
        # Calculating the length squared of the kth vector in our orthogonal set
        len_o_v_k = dot_product(orthogonals[k])
        # Calculating the length squared of the (k-1)th vector in our orthogonal set
        len_o_v_k1 = dot_product(orthogonals[k-1])
        # Calculating our lovasz coefficient exactly as described in the text
        lovasz_coeff = 3/4 - (dot_product(reduced[k],orthogonals[k-1])/dot_product(orthogonals[k-1]))**2
        # Checking if this length of the kth orthogonal vector is greater than or equal to the length of the 
        # (k-1)th orthogonal vector multiplied by our lovasz coefficient
        if len_o_v_k >= len_o_v_k1*lovasz_coeff:
            # If so, increment k
            k += 1
        else:
            # Otherwise, swap the kth and the (k-1)th element of our resulting matrix
            reduced[k], reduced[k-1] = reduced[k-1], reduced[k]
            # Set k to be the max of k-1 and 1
            k = max(k-1, 1)
    # Reduce based on front coefficients
    reduced = reducedBasis(reduced)
    # Return our resulting vector basis
    return reduced

print()
print("Problem b:")
print("Initial basis:")
basis = [[19,2,32,46,3,33],
         [15,42,11,0,3,24],
         [43,15,0,24,4,16],
         [20,44,44,0,18,15],
         [0,48,35,16,31,31],
         [48,33,32,9,1,29]]
for vtr in basis:
    print(vtr)
reduced_basis = LLL(basis)
print("\nReduced basis:")
for vtr in reduced_basis:
    print(vtr)

# I know I have done this correctly as this is the same answer as given in the text


Problem a:
b1 = [-63, 61]
b2 = [97, 83]
Final m value : -0.1362808842652796 ≈ 0
|v1 * v2| / ||v1||^2 = -0.1362808842652796 <= 1/2

Problem b:
Initial basis:
[19, 2, 32, 46, 3, 33]
[15, 42, 11, 0, 3, 24]
[43, 15, 0, 24, 4, 16]
[20, 44, 44, 0, 18, 15]
[0, 48, 35, 16, 31, 31]
[48, 33, 32, 9, 1, 29]

Reduced basis:
[7, -12, -8, 4, 19, 9]
[-20, 4, -9, 16, 13, 16]
[5, 2, 33, 0, 15, -9]
[-6, -7, -20, -21, 8, -12]
[-10, -24, 21, -15, -6, -11]
[7, 4, -9, -11, 1, 31]


# Problem 2

We're going to do some of the steps of the difference-of-squares factorization of the number $n=1424297947$. 

That is:  
- choose a value of $B$ appropriately, 
- find a collection of numbers $a_i$, each larger than $\sqrt{n}$, such that $b_i$ $a_i^2 - N$ are $B$-smooth,
- find an appropriate combination of the numbers $a_i$  such that the corresponding combination of the numbers $b_i$ is a perfect square, different than $a_i^2$
- reduce everything mod $n$, to find two numbers $a \not \equiv b \pmod{n}$ such that $a^2 \equiv b^2 \pmod{n}$. 

$n$ is small enough that you don't need to set up the full quadratic sieve, or really optimize anything in a serious way.

To do the mod 2 linear algebra: It's not the worst thing it the world to just write it yourself, or to find someone else on the internet who has done it.  Alternatively, it looks like sympy has a relatively new "DomainMatrix" in it, which will do the hard work for you.  For instance:



In [11]:
import math
import random
from sympy.polys.matrices import DomainMatrix
from sympy import GF
from sympy import Matrix
import numpy as np
from scipy.linalg import null_space


def ln(x):
    return math.log(x)

def sqrt(x):
    return math.sqrt(x)

# Function to find B given x
def findB(x):
    # Creating the exponent in the L(x) equation
    exponent = sqrt(ln(x) * ln(ln(x)))
    # Raising e to that exponent * 1/sqrt(2) so B = L(x)**(1/sqrt(2))
    B = math.e ** (exponent*(1 / sqrt(2)))
    # Returning floor so that the number returned is an integer
    return round(B)

def small_factorer(number, B):
    # Creating an array to store the factors
    factors = []
    # iterating up to the upper index
    for i in range(B):
        # Computing index
        index = i+1
        # Base case 
        if (index == 1):
            continue
        # If the remainder of the number divided by the index is zero
        if (number%index==0):
            # Set a count variable to zero
            count = 0
            while 1:
                # Divide the number by the index
                number//=index
                # Increment count
                count += 1
                # If no longer divisible then break
                if number%index!=0:
                    break     
            # Append factors with the index and count 
            factors.append([index, count])
    # If no small factors found, return 0
    if (number!=1):
        return 0
    else:
        # return discovered factors
        return factors
    
# This function serves to find a collection of a values so that a^2 = c (mod n)
# where c is the product of primes less than or equal to B
def find_a_collection(x, B):
    # calculating the integer square root of the modulus to generate random numbers over the range
    # root -> x
    # This is needed to ensure a^2 = c (without modulus applied)
    root = math.isqrt(x)
    # Initializing lists to hold our found a and c values
    a_list = []
    c_list = []
    # Creating a list to store all sets of factors found, this will be a matrix
    factor_list = []
    # Infinite while loop that will be broken from
    while 1:
        # Generating a random number from sqrt(x) -> x
        a = random.randint(root, x)
        # c = a^2 mod x
        c = a**2%x
        # Factoring c into small factors
        factors = small_factorer(c, B)
        # Making sure c factors into small factors
        if (factors != 0):
            # Appending a and c to their respective lists if c factors into small primes
            a_list.append(a)
            c_list.append(c)
            # Appending the factor list with the small factors of c
            factor_list.append(factors)
        # Breaking from the loop once the length of a values found is 10
        # This is realitively but experimental data showed me this number usually works
        if len(a_list) == 10:
            break
    # Returning found values
    return a_list, c_list, factor_list

# Unused function so I will not explain
def find_squares(factors):
    for index in range(len(factors)):
        truth = 1
        for factor in factors[index]:
            if factor[1]%2 != 0:
                truth = 0
        if truth:
            return index
    return -1

# This function finds the null space of the factor set found
def reduction(factor_set):
    # Initializing an array to hold all found factors
    factors_found = []
    # Iterating over factor set
    for factors in factor_set:
        # Iterating over factors for each c found
        for factor in factors:
            # Creating a variable to hold the factor
            f = factor[0]
            # If this factor is not already in the set, append it
            if f not in factors_found:
                factors_found.append(f)
    # Sort the set to make more readable
    factors_found.sort()
    # Creating an array where the columns correspond to the number of c values found
    # The number of rows here correspond to the number of factors found
    array = [[0 for j in range(len(factor_set))] for i in range(len(factors_found))]
    # Iterating over the number of c values found
    for index in range(len(factor_set)):
        factors = factor_set[index]
        # Iterating over the factors of c
        for factor in factors:
            f = factor[0]
            exp = factor[1]
            # Getting the index in the array of factors found
            f_ind = factors_found.index(f)
            # Storing the exponent of the factor in factor index row and cth column
            array[f_ind][index] = exp
    # Modular arithmatic function provided
    F = GF(2)
    entries = array
    entries_mod2 = [list(map(F, row)) for row in entries]
    M = DomainMatrix(entries_mod2, (len(factors_found),len(factor_set)), GF(2))
    # Calculating nullspace
    M = M.nullspace()
    M = M.to_Matrix()
    # Converting this to a list to make the data workable
    vtrs = M.tolist() 
    return vtrs

# GCDEX function used in previous assignments
def gcdex(a, b):
    # Base Case
    if a == 0 :
        return b,0,1
             
    gcd,x1,y1 = gcdex(b%a, a)
     
    # Update x and y using results of recursive
    # call
    x = y1 - (b//a) * x1
    y = x1
     
    return gcd, x, y

# Function to find a number so that a^2 = c^2 mod n
def diff_of_squares(n):
    # Finding B
    B = findB(n)
    # Getting values from find a collection function
    a_list, c_list, factors  = find_a_collection(n, B)
    # Getting nullspace
    null_space = reduction(factors)
    # If dimension of nullspace = 0, recursively call function to try again
    if len(null_space) == 0:
        return diff_of_squares(n)
    # Otherwise
    else:
        # Selecting a vector from the nullspace
        null_vtr = null_space[0]
        # Computing length of this vector
        length = len(null_vtr)
        # Initializing a and c product variables
        a_product = 1
        c_product = 1
        indexes = []
        # Iterating over the length
        for index in range(length):
            # Appending indices of nonzero elements in null space 
            if null_vtr[index]:
                indexes.append(index)
        # Iterating over indexes found
        for index in indexes:
            # Multiplying corresponding index in our a and c list to product variables
            a_product *= a_list[index]
            c_product *= c_list[index]
        # Print n
        print("n =", n)
        # Print c^2 without modulus
        print("c^2 =", c_product)
        # Calculate c by applying modulus then taking square root
        c = sqrt(c_product%n)
        # Print c
        print("c =", c)
        # Calculating a^2 without modulus
        a_squared = (a_product**2)
        # Print a^2 without modulus
        print("a^2 =", a_squared)
        # Calculate a by applying modulus then taking square root
        a = sqrt(a_squared%n)
        # Printing a
        print("a =", a)
        return a, c
    
    
    
            
n = 1424297947
diff_of_squares(n)

n = 1424297947
c^2 = 328406884
c = 18122.0
a^2 = 1256554572386102500
a = 18122.0


(18122.0, 18122.0)

# Problem 3

Here are some problems with cryptosystems - some version of each of which has happened in the real world, amazingly.

(i) What can go wrong if the RSA encryption exponent is a small number, like 3?  (This is actually a relatively common problem - small encryption exponents are often used for digital signature schemes in things like smart card readers, to keep the arithmetic simple and thus keep power requirements low).

(ii) Suppose N people set up RSA with two 256 bit primes (i.e. if you write the number in binary, it's 256 binary digits long).  Approximate the probability that one of these primes is used by two different people.  (Again, this has happened - alowing a key recovery attack by taking GCDs).

(iii) What goes wrong with ElGamal if, when sending a message, Bob chooses a predictable ephemeral key/nonce (say, by forgetting to initialize your pseudrandom number generator in a random way)?

(iv) What can go wrong if Alice sets up RSA twice, using the same public key $n=pq$, but two different public encryption exponents $e, e'$? 

i) For the most part, encryption with a relatively small exponent is relatively safe. One potential break in the algorithm could be a non-modular e^th root attack. If e is a relatively low-value, taking the cubic root of the published c would potentially give your x, depending on if x is less than the cubic root of x. Another pitfall of this is that the message could easily be recovered if the same message is encrypted with the same encryption exponent with using exponent number of setups with distinct public keys. Lets say e = 3 in this case, generating public keys N1, N2, N3, and yielding C1, C2, C3. We could then recover our message by first applying the Chinese Remainder Thm to C1, C2, C3 working under the modulus N1*N2*N3, which lets say generates C0. We know our x^3 < N1*N2*N3 therefore we can take the cubic root of our generated C0 to get exactly x.

ii) We see that a binary number with 256 digits has a max length of 2^255+1. We also know that the distribution of primes less than a given number is modeled by pi(N) which can be approximated by N / log(N).
We see that 2^255+1 / log ( 2^255+1) is about 3.2755 × 10^74. We can also see that the likelihood of selecting two numbers from this set and having these numbers be the same is 1/(3.2755*10^74)^2 which is about equal to 
9.3204 × 10^-150. This is an incredibly unlikely probabilty however it is not completely zero so it theoretically could happen.

iii) If Bob chooses a predictable nonce for his Elgamal encryption scheme, then k could easily be recovered. This would mean the discrete log problem would be solved for c1, giving us an accurate value of k. We could then calculate the inverse of k by running the extended Euclidean Algorithm on k and p-1, which is known as p and g are published values. Once the inverse of k is found. We could then raise A to that power and multiply this by c2. We know c2 = m*A^k, therefore if we multiply, we get m*A^k*A^(-k) = m*A. We know A therefore we can divide this value by A and get the message we want to recover.

iiii) Let us consider what happens if Eve runs RSA twice. Lets say the first time the algorithm is run, we get x^e1 = c1 and the second time we get x^e2 = c2, both working under a large modulus of n. A possible exploit here would occur here if we calculate GCD(e1, e2) = 1. This means that a linear combination of e1 and e2 is equal to 1. We could find said linear combination by running the extended Euclidean Algorithm to get w1 and w2 such that w1*e1 + w2*e2 = 1. We know that both e1 and e2 are relatively prime to (p-1)(q-1). We now see that if we take c1^w1 and c2^w2, we get x^(e1*w1) and x^(e2*w2). If we multiply these values together, we get x^(e1*w1) * x^(e2*w2) = x^(e1*w1+e2*w2) = x^1 by definition. This solves the RSA encrytption algorithm as we have now recovered our message x.

